# 10차시 : 고객정보 병합 및 파일 비교

---

## 1. 고객정보 병합

#### 고객기본정보.xlsx 파일과 고객부가정보.xlsx 파일을 고객번호 기준으로 병합

- 두 데이터프레임을 Key 컬럼값을 기준으로 병합
- 중복된 Key 값이 있으면 이상 데이터들이 생성되므로 중복제거
- 두 데이터프레임의 Key 컬럼명은 서로 동일

### 1.1 중복 고객정보 제거

In [ ]:
import pandas as pd

In [ ]:
고객기본정보 = pd.read_excel('../데이터/고객기본정보.xlsx')

In [ ]:
고객기본정보[:5]

#### len함수를 이용하여 데이터프레임 행수 확인

In [ ]:
len(고객기본정보)

#### unique함수를 이용하여 고객번호 컬럼의 유일한 값 목록 확인

In [ ]:
pd.unique(고객기본정보.고객번호)

#### len함수를 이용하여 고객번호 컬럼의 유일한 값의 갯수 확인

In [ ]:
len(pd.unique(고객기본정보.고객번호))

#### duplicated 함수를 이용하여 중복 고객정보 확인

- 데이터프레임내 Key 컬럼을 기준으로 중복된 값 확인
- duplicated 함수 사용
- 데이터프레임명.duplicated('Key컬럼명', keep='first 혹은 last') 형태로 사용
- first : 처음의 값을 제외하고 그 이후 값들이 중복임
- last : 마지막 값을 제외하고 그 이후 값들이 중복임
- 중복을 제외하기 위해 부정연산자인 ~ 사용

In [ ]:
고객기본정보[(고객기본정보.고객번호=='C001') | (고객기본정보.고객번호=='C007')]

In [ ]:
고객기본정보[고객기본정보.duplicated('고객번호', keep='last')]

#### 중복을 제외하고 중복되지 않은 값들만 저장

In [ ]:
고객기본정보[~고객기본정보.duplicated('고객번호', keep='last')]

In [ ]:
중복제거고객기본정보 = 고객기본정보[~고객기본정보.duplicated('고객번호', keep='last')]

In [ ]:
중복제거고객기본정보

#### 인덱스 재생성

In [ ]:
중복제거고객기본정보 = 중복제거고객기본정보.reset_index(drop = True)

In [ ]:
중복제거고객기본정보

In [ ]:
중복제거고객기본정보 = 중복제거고객기본정보.sort_values('고객번호', ascending=True)
중복제거고객기본정보

---

### 1.2 데이터프레임 컬럼명 변경

In [ ]:
고객부가정보 = pd.read_excel('../데이터/고객부가정보.xlsx')

In [ ]:
고객부가정보[:3]

#### 고객부가정보 Key컬럼명 변경
- 데이터프레임명.rename(columns={'변경전컬럼명':'변경후컬럼명'}, inplace=True)
- columns는 딕셔너리 값으로 지정되므로 여러 컬럼명을 동시에 변경 가능
- inplace = True 는 즉시 변경되도록 하기 위해 반드시 필요

In [ ]:
고객부가정보.rename(columns={'고객ID' :'고객번호'}, inplace=True)

In [ ]:
고객부가정보[:3]

### 1.3 고객정보 병합

- pd.merge(데이터프레임1, 데이터프레임2, on=['Key컬럼명'])
- Key 컬럼을 기준으로 데이터들이 연결되어 새로운 데이터프레임 생성

In [ ]:
고객정보병합 = pd.merge(중복제거고객기본정보, 고객부가정보, on=['고객번호'])

In [ ]:
고객정보병합

---

## 2. 고객정보 파일 비교

### 1.1 데이터프레임 인덱스 변경

- 데이터프레임의 인덱스 번호는 0부터 시작
- 엑셀 행번호는 2부터 시작
- 라인번호 일치를 위해 데이터프레임 인덱스 번호를 2부터 시작하도록 변경

In [ ]:
고객정보_V1 = pd.read_excel('../데이터/고객정보_V1.xlsx')

In [ ]:
고객정보_V2 = pd.read_excel('../데이터/고객정보_V2.xlsx')

In [ ]:
고객정보_V1[:3]

In [ ]:
고객정보_V2[:3]

#### 데이터프레임 인덱스 번호 2로 시작하도록 변경

In [ ]:
고객정보_V1.index

In [ ]:
고객정보_V1.index = 고객정보_V2.index + 2
고객정보_V1.index 

In [ ]:
고객정보_V2.index

In [ ]:
고객정보_V2.index = 고객정보_V2.index + 2
고객정보_V2.index 

#### 변경된 인덱스 번호 확인

In [ ]:
고객정보_V1[:3]

In [ ]:
고객정보_V2[:3]

---

### 2.2 변경된 라인과 컬럼 위치 찾기

- 두 데이터프레임 비교하여 변경된 경우 True, 변경되지 않은 경우 False
- stack 함수를 이용하여 변경여부 데이터를 reshape
- 변경된 위치의 라인번호와 컬럼명 찾기

In [ ]:
(고객정보_V1 != 고객정보_V2)

In [ ]:
스택 = (고객정보_V1 != 고객정보_V2).stack()

In [ ]:
스택

In [ ]:
변경라인컬럼 = 스택[스택]

In [ ]:
변경라인컬럼

In [ ]:
변경라인컬럼.index.names = ['라인번호', '컬럼명']

In [ ]:
pd.DataFrame(변경라인컬럼)

---

### 2.3 변경된 위치의 컬럼 값 비교

In [ ]:
import numpy as np

#### numpy를 이용하여 변경된 위치를 행/렬 정보 찾기
- numpy.where() 함수는 데이터프레임 내에서 True인 위치 정보를 추출

In [ ]:
변경위치 = np.where(고객정보_V1 != 고객정보_V2)

In [ ]:
변경위치

#### 두 데이터프레임의 변경된 위치의 컬럼값 저장

In [ ]:
V1 = 고객정보_V1.values[변경위치]
V1

In [ ]:
V2 = 고객정보_V2.values[변경위치]
V2

#### 두 데이터프레임의 변경된 위치 컬럼값으로 데이터프레임 생성

- pd.DataFrame([리스트])
- pd.DataFrame({'컬럼명': 리스트, '컬럼명': 리스트})

In [ ]:
변경된컬럼값 = pd.DataFrame({'고객정보_V1': V1, '고객정보_V2': V2})
변경된컬럼값

#### 변경된 라인 컬럼 정보와 변경된 컬럼 값 병합

In [ ]:
변경된컬럼값.index = 변경라인컬럼.index

In [ ]:
변경된컬럼값